In [ ]:
%load_ext autoreload
%autoreload 2

In [76]:
from jax import vmap
import jax.numpy as jnp
from pathlib import Path
import numpy as np
from einops import rearrange
import jax
import matplotlib.pyplot as plt
from jax.experimental.ode import odeint
from colora.data import load_all_hdf5, split_data_by_mu, prepare_coordinate_data
data_dir = Path('./data')
data_path = data_dir / 'burgers'
mus, sols, spacing = load_all_hdf5(data_path)

train_mus = np.asarray([0.001, 0.00199, 0.00298, 0.00496, 0.00595, 0.00694, 0.00892, 0.01])
test_mus = np.asarray([0.00397, 0.00793])
train_sols, test_sols = split_data_by_mu(mus, sols, train_mus, test_mus) # mus X variables X time X space_x X space_y
n_mu_train, n_q, n_t, n_x1, n_x2 = train_sols.shape
n_mu_test, n_q, n_t, n_x1, n_x2 = test_sols.shape
time = spacing[1]
x_space = spacing[2]
y_space = spacing[3]

print(f'n_mu train: {n_mu_train}, n_mu test: {n_mu_test}, n_variables: {n_q}, n_time samples: {n_t}, n_x samples: {n_x1}, n_x2 samples: {n_x2}')
# print(time)
# print(x_space)
# print(y_space)

y_train, mu_t_train, X_grid =  prepare_coordinate_data(spacing, train_mus, train_sols)
y_test, mu_t_test, X_grid =  prepare_coordinate_data(spacing, test_mus, test_sols)

def normalize(x, mean, std):
    return (x-mean)/std

mean, std = jnp.mean(mu_t_train, axis=0), jnp.std(mu_t_train, axis=0)
# print(mu_t_train.shape)
# print(mu_t_train)
mu_t_train = normalize(mu_t_train, mean, std)
mu_train = mu_t_train[:, 0]
print(mu_train)
mu_t_test = normalize(mu_t_test, mean, std)

n_mu train: 8, n_mu test: 2, n_variables: 1, n_time samples: 51, n_x samples: 129, n_x2 samples: 129
[-1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322
 -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322
 -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322
 -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322
 -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322
 -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322
 -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322
 -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322  -1.4352322
 -1.4352322  -1.4352322  -1.4352322  -1.1080291  -1.1080291  -1.1080291
 -1.1080291  -1.1080291  -1.1080291  -1.1080291  -1.1080291  -1.1080291
 -1.1080291  -1.1080291  -1.1080291  -1.1080291  -1.1080291  -1.1080291
 -1.1080291  -1.1080291  -1.1080291  -1.1080291  -1.1080291  -1.1080291
 -1.1080291  -1.1080291  -1.1080291

In [77]:
from colora.build import build_colora

key = jax.random.PRNGKey(1)

x_dim = 2
mu_t_dim = 2
u_dim = 1

d_len_x = x_space[-1] - x_space[0]
d_len_y = y_space[-1] - y_space[0]


u_layers = ['P', 'C', 'C', 'C', 'C', 'C', 'C', 'C'] # seven colora layers with 1 alpha each means we will have laten dim of 7
g_layers = ['D', 'D', 'D'] # takes in phi(mu,t) and outputs the time derivative of phi, i.e. the time derivative of phi1,...,phi7
rank = 3

u_hat_config = {'width': 25, 'layers': u_layers}
g_config = {'width': 15, 'layers': g_layers}

u_hat_fn, g_fn, theta_init, psi_init = build_colora(
    u_hat_config, g_config, x_dim, mu_t_dim, u_dim, lora_filter=['alpha'],period=[d_len_x, d_len_y], rank=rank, key=key)

#check neural ODE is correctly defined
print(g_fn)

<bound method Module.apply of DNN(
    # attributes
    width = 15
    layers = ['D', 'D', 'D']
    out_dim = 7
    activation = silu
    period = None
    rank = 1
    full = False
)>


In [82]:
# now we define the initial conditions of phi1,...,phi7 and integrate the neural ODE
n_phi = len(u_layers)-1
print(n_phi)
phi_init_wrt_mu = lambda mu: jnp.ones(n_phi)*mu#jnp.linspace(mu, mu+1, n_phi)
# then we use odeint to integrate g over the given list of times, with initial conditions phi_init_wrt_mu(mu)
# we will get phi(mu, t) for all mu and t
# we will use this to calculate u_hat
y=lambda 
def int_g()
    phi_mu= odeint(g_fn, phi_init_wrt_mu(mu), mu_t_train, , rtol=1e-6, atol=1e-6)

print(time)

7
<function <lambda> at 0x178e72de0>
[0.   0.02 0.04 0.06 0.08 0.1  0.12 0.14 0.16 0.18 0.2  0.22 0.24 0.26
 0.28 0.3  0.32 0.34 0.36 0.38 0.4  0.42 0.44 0.46 0.48 0.5  0.52 0.54
 0.56 0.58 0.6  0.62 0.64 0.66 0.68 0.7  0.72 0.74 0.76 0.78 0.8  0.82
 0.84 0.86 0.88 0.9  0.92 0.94 0.96 0.98 1.  ]


In [ ]:
u_hat_v_x =  vmap(u_hat_fn, in_axes=(None, None, 0)) # vmaped over x to generate solution field over space points
u_hat_v_x_phi =  vmap(u_hat_v_x, in_axes=(None, 0, None)) # vmaped over x to generate solution field over space points


In [85]:
n_phi = len(u_layers)-1

""" subject to change """
phi_init = jnp.linspace(0, 1, n_phi)

"""""""problematic part"""
#first define integrating g from phi_init from 0 to t
def integrate_g(g_fn, phi_init, time, mu, X_grid):
    phi = odeint(g_fn, phi_init, time, mu, X_grid)
    return phi

# define time integrator for g(phi(t, mu), psi) and vmap over mu
def integrate_g_forall_mu(g_fn, phi_init, time, mu, X_grid):
    batched_integrator = vmap(odeint, in_axes=(None, None, None, 0, None))
    phi_forall_mu = batched_integrator(g_fn, phi_init, time, mu, X_grid)
    return phi_forall_mu

# we will now define loss interms of the params of u_hat and g
def predict(psi_theta,mu_t, X_grid):
    psi, theta = psi_theta
    phi = integrate_g_forall_mu(g_fn_vmap, phi_init, time, mu_t, X_grid)
    u_hat = u_hat_v_x_phi(theta, phi, X_grid)
    return u_hat

def loss_NODE(psi_theta,sols, mu_t, X_grid):
    pred=predict(psi_theta,mu_t, X_grid)
    loss = jnp.linalg.norm(
        sols - pred, axis=(1,2)) / jnp.linalg.norm(sols, axis=(1,2))
    return jnp.mean(loss)


In [84]:
#quick test
def g_fn(x,t):
    return 1.0
phi_init =0.0
time =jnp.linspace(1,10000,2)
odeint(g_fn, phi_init, time)

Array([   0.   , 9999.001], dtype=float32)

In [89]:
# initialize phi
n_phi = len(u_layers) - 1
phi_init = jnp.ones(n_phi)

# def integrate_g_vmap_mu(g_fn, phi_init, time, mu_t): # integrate g over all time for all mu
#     batched_integrator = vmap(odeint, in_axes=(None, None, None, 0))
#     phi_forall_mu = batched_integrator(g_fn, phi_init, time, mu_t)
#     return phi_forall_mu

# def predict(psi_theta, phi_init, time, mu_t, X_grid):
#     psi, theta = psi_theta
#     phi = integrate_g_vmap_mu(g_fn, phi_init, time, mu_t)
#     pred = u_hat_v_x_phi(theta, phi, X_grid)
#     return pred

def predict(g_fn, psi_theta, phi_init, time, mu_t, X_grid):
    psi, theta = psi_theta
    
    # Integrate over all mu values using vmap for batch processing
    batched_integrator = vmap(odeint, in_axes=(None, None, None, 0))
    phi_forall_mu = batched_integrator(g_fn, phi_init, time, train_mus)

    u_hat_v_x_phi = vmap(vmap(u_hat_fn, in_axes=(None, None,0)), in_axes=(None, 0, None))
    pred = u_hat_v_x_phi(theta, phi_forall_mu, X_grid)
    return pred


def loss_NODE(psi_theta, mu_t, sols, X_grid):
    pred = predict(psi_theta, time, mu_t, X_grid)

    print("Shapes - sols: {}, pred: {}".format(sols.shape, pred.shape))
    
    loss = jnp.linalg.norm(
        sols - pred, axis=(1, 2)) / jnp.linalg.norm(sols, axis=(1, 2))
    return jnp.mean(loss)


In [87]:
from colora.data import Dataset

# the dataset is just responsible for batching the data over the mu_t tensor 
# in order to aviod memory overflow on the GPU
dataset = Dataset(mu_t_train, X_grid, y_train, n_batches=15, key=key)
dataset = iter(dataset)
def args_fn():
    return next(dataset)


In [39]:
print(mu_t_test)

[[-0.4536229  -1.6984156 ]
 [-0.4536229  -1.630479  ]
 [-0.4536229  -1.5625424 ]
 [-0.4536229  -1.4946058 ]
 [-0.4536229  -1.4266691 ]
 [-0.4536229  -1.3587325 ]
 [-0.4536229  -1.2907958 ]
 [-0.4536229  -1.2228593 ]
 [-0.4536229  -1.1549226 ]
 [-0.4536229  -1.086986  ]
 [-0.4536229  -1.0190494 ]
 [-0.4536229  -0.95111275]
 [-0.4536229  -0.8831761 ]
 [-0.4536229  -0.81523955]
 [-0.4536229  -0.7473029 ]
 [-0.4536229  -0.67936623]
 [-0.4536229  -0.61142963]
 [-0.4536229  -0.543493  ]
 [-0.4536229  -0.4755563 ]
 [-0.4536229  -0.40761977]
 [-0.4536229  -0.33968312]
 [-0.4536229  -0.27174655]
 [-0.4536229  -0.20380989]
 [-0.4536229  -0.13587321]
 [-0.4536229  -0.06793666]
 [-0.4536229   0.        ]
 [-0.4536229   0.06793656]
 [-0.4536229   0.13587332]
 [-0.4536229   0.20380989]
 [-0.4536229   0.27174643]
 [-0.4536229   0.3396832 ]
 [-0.4536229   0.40761977]
 [-0.4536229   0.4755563 ]
 [-0.4536229   0.5434931 ]
 [-0.4536229   0.61142963]
 [-0.4536229   0.67936623]
 [-0.4536229   0.74730295]
 

In [88]:
from colora.adam import adam_opt

psi_theta = (psi_init, theta_init)
opt_psi_theta, loss_history = adam_opt(psi_theta, loss_NODE, args_fn, steps=100, learning_rate=5e-3, verbose=True) 
opt_psi, opt_theta = opt_psi_theta

print(opt_psi_theta)
print(opt_psi)
print(opt_theta)

pred = predict(opt_psi_theta, mu_t_test, X_grid)
pred = rearrange(pred, '(M T) (N1 N2) Q -> M Q T N1 N2', Q=n_q, T=n_t, N1=n_x1, N2=n_x2)

test_vec =  rearrange(test_sols, 'M Q T N1 N2 -> M (Q T N1 N2)')
pred_vec =  rearrange(pred, 'M Q T N1 N2 -> M (Q T N1 N2)')
rel_err = np.linalg.norm(test_vec - pred_vec, axis=1) / np.linalg.norm(test_vec, axis=1)
mean_rel_err = rel_err.mean()
print(f'Test mean relative error: {mean_rel_err:.2E}')
from colora.plot import imshow_movie

imshow_movie(pred[0][0], save_to='./img/burgers.gif', t=spacing[1], title='Burgers', tight=True, live_cbar=True, frames=85)

  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: predict() missing 2 required positional arguments: 'mu_t' and 'X_grid'

In [ ]:
print(opt_psi_theta)
print(opt_psi)
print(opt_theta)

In [ ]:
pred = predict(opt_psi_theta, mu_t_test, X_grid)
pred = rearrange(pred, '(M T) (N1 N2) Q -> M Q T N1 N2', Q=n_q, T=n_t, N1=n_x1, N2=n_x2)

test_vec =  rearrange(test_sols, 'M Q T N1 N2 -> M (Q T N1 N2)')
pred_vec =  rearrange(pred, 'M Q T N1 N2 -> M (Q T N1 N2)')
rel_err = np.linalg.norm(test_vec- pred_vec, axis=1)/np.linalg.norm(test_vec, axis=1)
mean_rel_err = rel_err.mean()
print(f'Test mean relative error: {mean_rel_err:.2E}')

In [1]:
from colora.plot import imshow_movie

imshow_movie(pred[0][0], save_to='./img/burgers.gif', t=spacing[1], title='Burgers', tight=True, live_cbar=True, frames=85)

phis = h_v_mu_t(opt_psi, mu_t_test)
phis = rearrange(phis, '(M T) D -> M T D', T=n_t)


NameError: name 'pred' is not defined

In [ ]:
from colora.plot import trajectory_movie
leg= []
for i in range(phis.shape[-1]):
    lstr =rf'$\phi_{i}$'
    leg.append(lstr)
trajectory_movie(phis[0], x=time, title='Burgers', ylabel=r'$\phi(t;\mu)$', legend=leg, save_to='./img/burgers_dynamics', frames=85)